In [1]:
import pytest
from top2vec.top2vec import Top2Vec
from sklearn.datasets import fetch_20newsgroups
import numpy as np
import tempfile
import tensorflow_hub as hub

/home/ducanh/Credit/Top2Vec/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-20 11:33:59.381527: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737347639.394384 1515853 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737347639.398120 1515853 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-20 11:33:59.412486: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-cr

In [2]:
file_path = 'datasets/20NG/train_texts.txt'

with open(file_path, 'r') as file:
    t20ng_documents = file.readlines()
t20ng_documents = [doc.strip() for doc in t20ng_documents]


In [3]:
file_path = 'datasets/20NG/test_texts.txt'
with open(file_path, 'r') as file:
    t20ng_documents_test = file.readlines()
t20ng_documents_test = [doc.strip() for doc in t20ng_documents_test]

In [4]:
vocab_path = 'datasets/20NG/vocab.txt'
with open(vocab_path, 'r') as file:
    vocab = file.readlines()
t20ng_vocabs = [v.strip() for v in vocab]

In [5]:
top2vec = Top2Vec.load("top2vec_model_2.ihb")

### calculate TD


In [6]:
top2vec.get_num_topics()

223

In [ ]:
len(t20ng_documents_test)

7532

In [9]:
theta = top2vec.get_theta(t20ng_documents_test)

Embedding documents: 100%|██████████| 236/236 [00:15<00:00, 14.77it/s]
7532it [00:00, 27971.56it/s]
Calculating document topic distributions: 100%|██████████| 7532/7532 [00:03<00:00, 2083.79it/s]


In [16]:
labels_path = 'datasets/20NG/test_labels.txt'

with open(labels_path, 'r') as file:
    labels = file.readlines()
t20ng_test_labels = np.array([int(label.strip()) for label in labels])

(7532,)


In [18]:
from topmost import eva
clustering_results = eva._clustering(theta, t20ng_test_labels)

In [22]:
top_words, score, topic_nums = top2vec.get_topics()

In [24]:
top_words[0]

array(['keywords nhl', 'hockey coverage', 'oilers playoffs',
       'hockey community', 'numbers nhl', 'philadelphia flyers',
       'professional hockey', 'bruins buffalo', 'ottawa senators',
       'central hockey', 'put puck', 'sharks review', 'hockey league',
       'nhl teams', 'passed puck', 'flyers fan', 'hockey champs',
       'usenet hockey', 'montreal ottawa', 'college hockey',
       'playoff hockey', 'season minnesota', 'hockey players',
       'louis blues', 'nhl expansion', 'capitals mike', 'games season',
       'wins stanley', 'oilers traded', 'stanley cup', 'buffalo sabres',
       'nhl playoff', 'win stanley', 'penguins devils',
       'nelson recsporthockey', 'wings win', 'ottawa ottawa',
       'playoff predictions', 'european hockey', 'washington capitals',
       'penguins games', 'hockey team', 'edmonton oilers', 'hockey pool',
       'pittsburgh penguins', 'calgary los', 'europeans nhl',
       'buffalo boston', 'canadian hockey', 'hockey fans'], dtype='<U26')

In [28]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from tqdm import tqdm
def calculate_diversity(topic_model):
    top_words, score, topic_nums = topic_model.get_topics(50)
    K = len(top_words)
    T = len(top_words[0])
    top_words_strings = []
    for topic_words in top_words:
        top_words_strings.append(" ".join(topic_words))
    vectorizer = CountVectorizer(tokenizer=lambda x: x.split())
    counter = vectorizer.fit_transform(top_words_strings).toarray()
    TF = counter.sum(axis=0)
    TD = (TF == 1).sum() / (K * T)
    return TD

td = calculate_diversity(top2vec)
td
    

0.2008

In [11]:
len(top2vec.topic_vectors)

223

In [20]:
from topmost import eva
def calculate_cv(topic_model, corpus, vocab):
    # calculate topic coherence
    top_words, score, topic_nums = topic_model.get_topics()
    top_words_strings = []
    for topic_words in top_words:
        top_words_strings.append(" ".join(topic_words))
    return eva._coherence(corpus, vocab, top_words_strings)

cv = calculate_cv(top2vec, t20ng_documents, t20ng_vocabs)
cv
    
    

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

0.49367605042932583